# Quantum Genetic Algorithm

## outline

1. generate statevector with uniform distribution
1. observe first candidate `u` (using measurement)
1. reconstruct statevector and apply one grover iteration using fitness
1. observe second candidate `w` (using measurement)
1. compare fitness of `u` and `w` and decides the winner
1. reconstruct statevector with updated amplitude using the winner
1. repeat steps 2 - 6 until statevector converges and observe the solution

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib inline

# importing Qiskit
from qiskit import Aer, IBMQ
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import compile
from qiskit.backends.ibmq import least_busy

from qiskit.tools import visualization
from qiskit.tools.visualization import circuit_drawer

In [ ]:
IBMQ.load_accounts()

We need to implement functions to create (reconstruct) statevector from amplitude list. And since almost every step of the algorithm needs a measurement, we'll need a boilerplate function for that.

In [ ]:
def setup_circuit(n):
    qs = QuantumRegister(n, 'state_vec')
    st = QuantumRegister(n, 'cand')
    anc = QuantumRegister(2*n-2, 'anc')
    out = QuantumRegister(1, 'out')
    cs = ClassicalRegister(n, 'cs')
    qc = QuantumCircuit(qs, st, anc, out, cs)
    return qc, qs, st, anc, out, cs

def set_state(qc, qs, angles):
    # TODO: maybe change angles to amplitude
    if len(qs) != len(angles):
        raise ValueError('number of qubits is not the same as number of angles')
    for i in range(len(qs)):
        qc.u3(angles[i], 0, 0, qs[i])
    qc.barrier(qs)
        
def execute_circuit(n, qc, shots=1024, simulation=True, id_msg='', log=False):
    if not simulation:
        backend = IBMQ.get_backend(filter=lambda x: x.configuration()['n_qubits'] > n and 
                                   not x.configuration()['simulator'] and x.status()['operational'] == True)
    else:
        backend = IBMQ.get_backend('ibmq_qasm_simulator')
    qobj = compile(qc, backend=backend, shots=shots)
    job = backend.run(qobj)

    # TODO: select result which appear the most
    lapse = 0
    interval = 5
    if log: print('running: {}'.format(id_msg))
    while job.status().name != 'DONE':
        if log:
            print('Status @ {} seconds'.format(interval * lapse))
            print(job.status())
            print('quere pos: {}'.format(job.queue_position()))
        time.sleep(interval)
        lapse += 1
    if log: print(job.status())
    result = job.result()
    counts = result.get_counts(qc)
    return max(counts, key=lambda k: counts[k])

Step 3 and 5 of the algorithm namely the grover application step and the compete step, will need a way to compare the result between the two candidate in a quantum mechanical way. Quantum Comparator is needed to accomplish this. 

So in the next step we will implement the quantum comparator and int2qubit function to initialize bitstring value. We will use n_control_not from q_util module so we won't need to implement it again.

In [ ]:
from utility import n_control_not

def q_comparator(qc, a, b, anc, target):
    # check whether two register is the same length
    if len(a) != len(b):
        raise ValueError('two register to compare must have the same length: a is {}, b is {}'.format(len(a), len(b)))
    
    anc_len = len(anc)
    n = len(a)
    if anc_len < 2 * n - 2:
        raise ValueError('ancillary bit is not enough: anc_len is {}, need at least {} qbit'.format(anc_len, len(a) - 1))
    
    # compare the MSB
    n_control_not(qc, [a[n-1], b[n-1]], target, anc, [1, -1])
    
    for i in range(n - 1):
        # all more significant bits must be equal
        j = n - 1 - i # last significant bit
        # j-1 is the bit we're comparing
        qc.ccx(a[j], b[j], anc[i])
        qc.x(a[j])
        qc.x(b[j])
        qc.ccx(a[j], b[j], anc[i])
        
        n_control_not(qc, [anc[k] for k in range(i + 1)] + [a[j-1], b[j-1]], target, 
                      [anc[k] for k in range(i+1, anc_len)],
                      [1] * (i + 2) + [-1])
        
    for j in range(1, n):
        qc.ccx(a[j], b[j], anc[n-1-j])
        qc.x(a[j])
        qc.x(b[j])
        qc.ccx(a[j], b[j], anc[n-1-j])

def int_to_qubit(n, circuit, qs):
    bits_required = int(np.log2(max(n,1)) + 1)
    qs_len = len(qs)
    if qs_len < bits_required:
        raise ValueError('input n = {} requires {} bits but qs is only {} bits'.format(n, bits_required, qs_len))

    bstr = '{0:b}'.format(n)
    bstr = bstr.zfill(qs_len)
    for i in range(qs_len):
        if bstr[qs_len-i-1] == '1':
            circuit.x(qs[i])
            
def bstr_to_qubit(bstr, circuit, qs):
    if len(qs) < len(bstr):
        raise ValueError('bstr in longer than register')
    for i in range(len(bstr)):
        if bstr[-1-i] == '1':
            circuit.x(qs[i])

Onto implementing the main algorithm

1. generate statevector with uniform distribution
1. observe first candidate `u` (using measurement)
1. reconstruct statevector and apply one grover iteration using fitness
1. observe second candidate `w` (using measurement)
1. compare fitness of `u` and `w` and decides the winner
1. reconstruct statevector with updated amplitude using the winner
1. repeat steps 2 - 6 until statevector converges and observe the solution

In [ ]:
from utility import Grover

def get_candidate(n, angles):
    qc, qs, st, anc, out, cs = setup_circuit(n)
    set_state(qc, qs, angles)
    qc.barrier()
    qc.measure(qs, cs)
    candidate = execute_circuit(n, qc)
    return candidate
    
    
def get_second_candidate(n, angles, first_candiate, fitness=q_comparator):
    qc, qs, st, anc, out, cs = setup_circuit(n)
    set_state(qc, qs, angles)
    bstr_to_qubit(first_candiate, qc, st)
    
    # prepare for grover
    qc.x(out)
    qc.h(out)
    # do Uf
    fitness(qc, qs, st, anc, out[0])
    # inv around mean
    Grover.inv_around_mean(qc, qs, anc)
    # execute circuit
    qc.barrier()
    qc.measure(qs, cs)
    second_candidate = execute_circuit(n, qc)
    return second_candidate
    
def quantum_genetic_algorithm(n, fitness=None):
    # init starting angles
    angles = [np.pi/2 for _ in range(n)]
    
    # condition for stopping the algorithm
    def is_converge(angs):
        for ang in angs:
            if np.pi/18 <= ang <= np.pi/18*17: return False
        return True
    
    def update_angles(angs, sol):
        step = 0.05
        angs = [angs[i] + step if sol[i] =='1' else angs[i] - step for i in range(len(sol))]
        return angs
    
    gen_count = 1
    while not is_converge(angles):
        f_cand = get_candidate(n, angles)
        s_cand = get_second_candidate(n, angles, f_cand)
        # TODO: truly competes and not cheating
        winner = max(f_cand, s_cand)
        angles = update_angles(angles, winner)
        print('generation @{}:'.format(gen_count))
        print('    f:{}, s:{}, angles:{}'.format(f_cand, s_cand, angles))
        gen_count += 1
    
    print('==============================================')
    print('finish')
    print('total generation:{}'.format(gen_count))
    print('solution: {}'.format(winner))

In [ ]:
quantum_genetic_algorithm(4)